importing libraries

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

load the tensorflow dataset

In [6]:
import tensorflow_datasets as tfds

load the data

In [7]:
(train_data, test_data), ds_info = tfds.load('tf_flowers',
                                             split=['train[:80%]', 'train[80%:]'],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True)

Preprocess the data

In [8]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

In [9]:
train_data = train_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

loading pre trained models ResNet50

In [10]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


-freezing the base model to retain its learned weights

In [11]:
base_model.trainable = False

-adding custom layers on top of the pre trained model

In [12]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(ds_info.features['label'].num_classes, activation='softmax')(x)

-building the model

In [13]:
model = Model(inputs=base_model.input, outputs=predictions)

compile the model

In [14]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

train the model

In [17]:
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=30)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.6191 - loss: 0.9880 - val_accuracy: 0.5163 - val_loss: 1.2404
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 9s 101ms/step - accuracy: 0.6184 - loss: 0.9836 - val_accuracy: 0.5204 - val_loss: 1.2162
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 9s 99ms/step - accuracy: 0.6238 - loss: 0.9758 - val_accuracy: 0.5177 - val_loss: 1.2114
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 109ms/step - accuracy: 0.6306 - loss: 0.9645 - val_accuracy: 0.5259 - val_loss: 1.2106
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 105ms/step - accuracy: 0.6385 - loss: 0.9653 - val_accuracy: 0.5191 - val_loss: 1.2204
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 9s 99ms/step - accuracy: 0.6322 - loss: 0.9592 - val_accuracy: 0.5286 - val_loss: 1.2114
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 105ms/step - accuracy: 0.6456 - loss: 0.9506 - val_accuracy: 0.5327 - val_loss: 1.1979
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - accuracy: 0.6382 - loss: 0.9451 - val_accuracy:

fine tuning dense layers

-unfreeze the layers for fine-tuning

In [18]:
base_model.trainable = True

-recompile the model with lower learning rate for fine tuning

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

-train the model again for fine-tuning

In [20]:
fine_tune_history = model.fit(train_data,
                              validation_data=test_data,
                              epochs=15)

Epoch 1/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 116s 641ms/step - accuracy: 0.3323 - loss: 80.5973 - val_accuracy: 0.1703 - val_loss: 48.1196
Epoch 2/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 28s 299ms/step - accuracy: 0.8309 - loss: 7.3167 - val_accuracy: 0.2575 - val_loss: 79.0216
Epoch 3/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 28s 299ms/step - accuracy: 0.9150 - loss: 2.3377 - val_accuracy: 0.2207 - val_loss: 105.3223
Epoch 4/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 42s 306ms/step - accuracy: 0.9651 - loss: 0.7228 - val_accuracy: 0.1703 - val_loss: 105.7842
Epoch 5/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 308ms/step - accuracy: 0.9754 - loss: 0.3987 - val_accuracy: 0.2316 - val_loss: 85.9193
Epoch 6/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 297ms/step - accuracy: 0.9962 - loss: 0.0571 - val_accuracy: 0.3297 - val_loss: 71.4617
Epoch 7/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 42s 307ms/step - accuracy: 0.9994 - loss: 0.0079 - val_accuracy: 0.4046 - val_loss: 55.5255
Epoch 8/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 28s 305ms/step - accuracy: 0.9925 - loss: 0.0820

evaluate the model

In [21]:
test_loss, test_acc = model.evaluate(test_data)

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step - accuracy: 0.8108 - loss: 9.6942


-get predictions

In [22]:
y_true = np.concatenate([y for x, y in test_data], axis=0)
y_pred = np.argmax(model.predict(test_data), axis=1)

23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 246ms/step


classification report

In [23]:
print(classification_report(y_true, y_pred, target_names=ds_info.features['label'].names))

              precision    recall  f1-score   support

   dandelion       0.86      0.86      0.86       159
       daisy       0.73      0.83      0.78       120
      tulips       0.85      0.76      0.80       186
  sunflowers       0.85      0.80      0.82       138
       roses       0.74      0.79      0.76       131

    accuracy                           0.81       734
   macro avg       0.81      0.81      0.81       734
weighted avg       0.81      0.81      0.81       734

